In [3]:

!pip install webdriver_manager


  Using cached webdriver_manager-3.4.2-py2.py3-none-any.whl (16 kB)
  Using cached configparser-5.0.2-py3-none-any.whl (19 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\backports\\__init__.py'
Consider using the `--user` option or check the permissions.



In [5]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import requests
# from webdriver_manager.chrome import ChromeDriverManager

In [13]:
# from webdriver_manager.chrome import ChromeDriverManager
# executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path, headless=False)
nyc_soup = BeautifulSoup(requests.get("https://www1.nyc.gov/site/finance/taxes/property-annualized-sales-update.page").text, "html.parser")

In [23]:
url_list = []

for tr in list(nyc_soup.find_all("tr"))[8:22]:
    url_list.append(f"https://www1.nyc.gov{tr.find_all('a')[1]['href']}")







In [27]:
list_df = []
for url in url_list:
    try:
        list_df.append(pd.read_excel(url))
    except:
        print(url)





https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/neighborhood_sales/2015_citywide_ sales.xlsx
https://www1.nyc.gov/assets/finance/downloads/pdf/rolling_sales/neighborhood_sales/2014_citywide_ sales.xls
